In [1]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
import numpy as np
from modules.env import MazeVisualization
from utils.skeletonize import skeletonize_env, draw_scatter_plot_skeleton
import seaborn as sns
from modules.base import SRMB
import matplotlib

import networkx as nx
import matplotlib.gridspec as gridspec
from matplotlib.ticker import MultipleLocator
from tests.robustness import postprocess_all_skel_gw, postprocess_all_skel_gw_nonisomorphic

%matplotlib inline
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['svg.fonttype'] = 'none'
matplotlib.rcParams['ps.fonttype'] = 42
rcParams['font.sans-serif'] = ['Arial']  
rcParams['font.family'] = 'sans-serif'

sns.set_style(
    'ticks',
    rc={
        'xtick.direction': 'out',  
        'ytick.direction': 'out',  
    }
)
rcParams["figure.dpi"] = 600
# Set the font to Arial
big_font = 20
mid_font = 15
small_font = 12

indicator_font = big_font
legend_font = mid_font
title_font = mid_font
axis_font = mid_font
tick_font = small_font



subplot_scale_factor = 3

indicator_x_loc = -0.1
indicator_y_loc = 1.15

node_size = 300
edge_width = 4


In [2]:
junction_color = "#416788"
deadend_color = "#646464"
edge_color = "#E1DAD2"

palette = ['#003f5c', '#58508d', '#bc5090', '#ff6361', '#ffa600']

In [3]:
from utils.ot import calc_ot_post_procrustes, center_and_scale_diffusionmap
from tests.robustness import interpolate_matrix_column_wise, reindex_matrix_column_wise, original_env_states_in_shifted_env, shift_topology_state_list
import copy

def process_shift(env):
    srmb = SRMB(env.num_row, env.num_column)
    mtx_original_list = []
    mtx_transformed_list = []
    srmb.update_map_structure(env.blocks)
    eigenmap = srmb.return_successormap()
    (
        vertex_nodes,
        deadend_nodes,
        edge_nodes,
        G,
        image,
        skeleton,
        vertex_corresp,
        deadend_corresp,
        edge_corresp,
        _, _, _
    ) = skeletonize_env(env)
    vertex_nodes_list = []
    deadend_nodes_list = []
    edge_nodes_list = []
    vertex_nodes_list.append(vertex_nodes)
    deadend_nodes_list.append(deadend_nodes)
    edge_nodes_list.append(edge_nodes)

    for row_shift, column_shift in [(0, 0), (0, 1), (1, 1), (0, 2), (1, 2), (2, 2)]:
        if row_shift == 0 and column_shift == 0:
            (
                mtx_original,
                mtx_transformed,
                mtx_stand,
                grid_cost_pot,
                grid_cost_disp,
                best_sign_configuration,
                best_optimal_rotation_matrix,
                scale,
                scale_shifted,
            ) = calc_ot_post_procrustes(eigenmap, eigenmap)
            mtx_original_list.append(center_and_scale_diffusionmap(mtx_stand[env.nonblocks, :3].real)[0])
            mtx_transformed_list.append(center_and_scale_diffusionmap(mtx_transformed[env.nonblocks, :3].real)[0])
            continue

        env_shifted = copy.deepcopy(env)
        env_shifted.shift_map_w_expansion((row_shift, column_shift))

        blocks = env_shifted.return_blocks()
        srmb_shifted = SRMB(env_shifted.num_row, env_shifted.num_column)
        srmb_shifted.update_map_structure(blocks=blocks)

        vertex_nodes_shifted = []
        for vertex_node_list in vertex_nodes:
            new_vertex_node_list = new_vertex_node_list = shift_topology_state_list(vertex_node_list, row_shift, column_shift, env, env_shifted)
            vertex_nodes_shifted.append(new_vertex_node_list)
        vertex_nodes_list.append(vertex_nodes_shifted)
        
        deadend_nodes_shifted = []
        for deadend_node_list in deadend_nodes:
            new_deadend_node_list = shift_topology_state_list(deadend_node_list, row_shift, column_shift, env, env_shifted)
            deadend_nodes_shifted.append(new_deadend_node_list)
        deadend_nodes_list.append(deadend_nodes_shifted)

        edge_nodes_list.append(edge_nodes)
        
        eigenmap_shifted = srmb_shifted.return_successormap()

        original_states = original_env_states_in_shifted_env(
            env, row_shift, column_shift
        )



        (
            mtx_original,
            mtx_transformed,
            mtx_stand,
            grid_cost_pot,
            grid_cost_disp,
            best_sign_configuration,
            best_optimal_rotation_matrix,
            scale,
            scale_shifted,
        ) = calc_ot_post_procrustes(eigenmap, eigenmap_shifted, original_states=original_states)
        mtx_original_list.append(center_and_scale_diffusionmap(mtx_stand[env_shifted.nonblocks, :3].real)[0])
        mtx_transformed_list.append(center_and_scale_diffusionmap(mtx_transformed[env_shifted.nonblocks, :3].real)[0])
    return mtx_original_list, mtx_transformed_list, vertex_nodes_list, deadend_nodes_list, edge_nodes_list

def process_scale(env, max_scale):
    srmb = SRMB(env.num_row, env.num_column)

    mtx_original_list = []
    mtx_transformed_list = []

    vertex_nodes_list = []
    deadend_nodes_list = []
    edge_nodes_list = []
    srmb.update_map_structure(env.blocks)
    (
        vertex_nodes,
        deadend_nodes,
        edge_nodes,
        G,
        image,
        skeleton,
        vertex_corresp,
        deadend_corresp,
        edge_corresp,
        _, _, _
    ) = skeletonize_env(env)
    vertex_nodes_list.append(vertex_nodes)
    deadend_nodes_list.append(deadend_nodes)
    edge_nodes_list.append(edge_nodes)

    eigenmap = srmb.return_successormap()


    for scale_factor in range(1, max_scale + 1):
        if scale_factor == 1:
            (
                mtx_original,
                mtx_transformed,
                mtx_stand,
                grid_cost_pot,
                grid_cost_disp,
                best_sign_configuration,
                best_optimal_rotation_matrix,
                scale,
                scale_scaled,
            ) = calc_ot_post_procrustes(eigenmap, eigenmap)
            mtx_original_list.append(mtx_stand[env.nonblocks, :3].real)
            mtx_transformed_list.append(center_and_scale_diffusionmap(mtx_transformed[env.nonblocks, :3].real)[0])
        else:
            env_scaled = copy.deepcopy(env)
            env_scaled.scale_map(scale_factor)

            vertex_nodes_scaled = []
            for vertex_node_list in vertex_nodes:
                new_vertex_node_list = []
                for v in vertex_node_list:
                    vr, vc = env.state_to_index(v)
                    for dvr in range(scale_factor):
                        for dvc in range(scale_factor):
                            new_vertex_node_list.append(
                                env_scaled.index_to_state(
                                    vr * scale_factor + dvr, vc * scale_factor + dvc
                                )
                            )
                vertex_nodes_scaled.append(new_vertex_node_list)

            deadend_nodes_scaled = []
            for deadend_node_list in deadend_nodes:
                new_deadend_node_list = []
                for d in deadend_node_list:
                    dr, dc = env.state_to_index(d)
                    for ddr in range(scale_factor):
                        for ddc in range(scale_factor):
                            new_deadend_node_list.append(
                                env_scaled.index_to_state(
                                    dr * scale_factor + ddr, dc * scale_factor + ddc
                                )
                            )
                deadend_nodes_scaled.append(new_deadend_node_list)

            edge_nodes_scaled = []
            for edge_node_list in edge_nodes:
                new_edge_node_list = []
                for e in edge_node_list:
                    er, ec = env.state_to_index(e)
                    for der in range(scale_factor):
                        for dec in range(scale_factor):
                            new_edge_node_list.append(
                                env_scaled.index_to_state(
                                    er * scale_factor + der, ec * scale_factor + dec
                                )
                            )
                edge_nodes_scaled.append(new_edge_node_list)

            vertex_nodes_list.append(vertex_nodes_scaled)
            deadend_nodes_list.append(deadend_nodes_scaled)
            edge_nodes_list.append(edge_nodes_scaled)

            vertex_corresp_scaled = []
            for vcr, vcc in vertex_corresp:
                vertex_corresp_scaled.append(
                    (vcr * scale_factor, vcc * scale_factor)
                )
            vertex_corresp_scaled = tuple(vertex_corresp_scaled)

            deadend_corresp_scaled = []
            for dcr, dcc in deadend_corresp:
                deadend_corresp_scaled.append(
                    (dcr * scale_factor, dcc * scale_factor)
                )
            deadend_corresp_scaled = tuple(deadend_corresp_scaled)

            blocks = env_scaled.return_blocks()
            srmb_scaled = SRMB(env_scaled.num_row, env_scaled.num_column)
            srmb_scaled.update_map_structure(blocks=blocks)
            eigenmap_scaled = srmb_scaled.return_successormap()
            eigenmap_interpolated = interpolate_matrix_column_wise(
                eigenmap, scale_factor, env
            )

            (
                mtx_original,
                mtx_transformed,
                mtx_stand,
                grid_cost_pot,
                grid_cost_disp,
                best_sign_configuration,
                best_optimal_rotation_matrix,
                scale,
                scale_scaled,
            ) = calc_ot_post_procrustes(eigenmap_interpolated, eigenmap_scaled)
            mtx_original_list.append(
                mtx_stand[env_scaled.nonblocks, :3].real
            )
            mtx_transformed_list.append(
                center_and_scale_diffusionmap(mtx_transformed[env_scaled.nonblocks, :3].real)[0]
            )
    return mtx_original_list, mtx_transformed_list, vertex_nodes_list, deadend_nodes_list, edge_nodes_list


def process_rotation(env):
    max_rotation = 4
    srmb = SRMB(env.num_row, env.num_column)


    mtx_original_list = []
    mtx_transformed_list = []
    vertex_nodes_list = []
    deadend_nodes_list = []
    edge_nodes_list = []
    srmb.update_map_structure(env.blocks)
    (
        vertex_nodes,
        deadend_nodes,
        edge_nodes,
        G,
        image,
        skeleton,
        vertex_corresp,
        deadend_corresp,
        edge_corresp,
        _, _, _
    ) = skeletonize_env(env)
    vertex_nodes_list.append(vertex_nodes)
    deadend_nodes_list.append(deadend_nodes)
    edge_nodes_list.append(edge_nodes)

    eigenmap = srmb.return_successormap()
    for rotation_factor in range(max_rotation):
        if rotation_factor == 0:
            (
                mtx_original,
                mtx_transformed,
                mtx_stand,
                grid_cost_pot,
                grid_cost_disp,
                best_sign_configuration,
                best_optimal_rotation_matrix,
                scale,
                scale_rotated,
            ) = calc_ot_post_procrustes(eigenmap, eigenmap)
            mtx_original_list.append(center_and_scale_diffusionmap(mtx_stand[env.nonblocks, :3].real)[0])
            mtx_transformed_list.append(center_and_scale_diffusionmap(eigenmap[env.nonblocks, :3].real)[0])

        else:
            env_rotated = copy.deepcopy(env)
            vertex_nodes_rotated = []
            for vertex_node_list in vertex_nodes:
                new_vertex_node_list = []
                for v in vertex_node_list:
                    vr, vc = env.state_to_index(v)
                    for _ in range(rotation_factor):
                        nvr = env.num_row - 1 - vc
                        nvc = vr
                        vr, vc = nvr, nvc
                    new_vertex_node_list.append(env_rotated.index_to_state(vr, vc))
                vertex_nodes_rotated.append(new_vertex_node_list)

            deadend_nodes_rotated = []
            for deadend_node_list in deadend_nodes:
                new_deadend_node_list = []
                for d in deadend_node_list:
                    dr, dc = env.state_to_index(d)
                    for _ in range(rotation_factor):
                        ndr = env.num_row - 1 - dc
                        ndc = dr
                        dr, dc = ndr, ndc
                    new_deadend_node_list.append(env_rotated.index_to_state(dr, dc))
                deadend_nodes_rotated.append(new_deadend_node_list)

            edge_nodes_rotated = []
            for edge_node_list in edge_nodes:
                new_edge_node_list = []
                for e in edge_node_list:
                    er, ec = env.state_to_index(e)
                    for _ in range(rotation_factor):
                        ner = env.num_row - 1 - ec
                        nec = er
                        er, ec = ner, nec
                    new_edge_node_list.append(env_rotated.index_to_state(er, ec))
                edge_nodes_rotated.append(new_edge_node_list)

            vertex_nodes_list.append(vertex_nodes_rotated)
            deadend_nodes_list.append(deadend_nodes_rotated)
            edge_nodes_list.append(edge_nodes_rotated)

            vertex_corresp_rotated = []
            for vcr, vcc in vertex_corresp:
                for _ in range(rotation_factor):
                    nvr = env.num_row - 1 - vcc
                    nvc = vcr
                    vcr, vcc = nvr, nvc
                vertex_corresp_rotated.append((vcr, vcc))
            vertex_corresp_rotated = tuple(vertex_corresp_rotated)

            deadend_corresp_rotated = []
            for dcr, dcc in deadend_corresp:
                for _ in range(rotation_factor):
                    ndr = env.num_row - 1 - dcc
                    ndc = dcr
                    dcr, dcc = ndr, ndc
                deadend_corresp_rotated.append((dcr, dcc))
            deadend_corresp_rotated = tuple(deadend_corresp_rotated)

            env_rotated.rotate_map(rotation_factor)

            blocks = env_rotated.return_blocks()
            srmb_rotated = SRMB(env_rotated.num_row, env_rotated.num_column)
            srmb_rotated.update_map_structure(blocks=blocks)
            eigenmap_rotated = srmb_rotated.return_successormap()

            eigenmap_reindexed = reindex_matrix_column_wise(
                eigenmap, rotation_factor, env
            )
            (
                mtx_original,
                mtx_transformed,
                mtx_stand,
                grid_cost_pot,
                grid_cost_disp,
                best_sign_configuration,
                best_optimal_rotation_matrix,
                scale,
                scale_rotated,
            ) = calc_ot_post_procrustes(eigenmap_reindexed, eigenmap_rotated)
            mtx_original_list.append(center_and_scale_diffusionmap(mtx_stand[env_rotated.nonblocks, :3].real)[0])
            mtx_transformed_list.append(center_and_scale_diffusionmap(mtx_transformed[env_rotated.nonblocks, :3].real)[0])
    return mtx_original_list, mtx_transformed_list, vertex_nodes_list, deadend_nodes_list, edge_nodes_list


def generate_seed_numbers(num_seeds):
    # Set an initial seed for reproducibility
    np.random.seed(7)
    
    # Define the range for seed numbers
    max_seed_value = 1000000
    
    # Ensure num_seeds does not exceed the range
    if num_seeds > max_seed_value:
        raise ValueError("num_seeds exceeds the maximum number of unique seeds available.")
    
    # Generate a fixed set of unique seed numbers
    seed_numbers = np.random.choice(range(max_seed_value), size=num_seeds, replace=False)
    
    return seed_numbers


def process_noise(
    env, num_sample_per_env, min_ratio=0.01, max_ratio=0.2
):
    srmb = SRMB(env.num_row, env.num_column)
    mtx_original_list = []
    mtx_transformed_list = []
    srmb.update_map_structure(env.blocks)
    vertex_nodes_list = []
    deadend_nodes_list = []
    edge_nodes_list = []
    env_list = []

    (
        vertex_nodes,
        deadend_nodes,
        edge_nodes,
        G,
        image,
        skeleton,
        vertex_corresp,
        deadend_corresp,
        edge_corresp,
        _, _, _
    ) = skeletonize_env(env)
    if G is None:
        return 0

    eigenmap = srmb.return_successormap()

    seed_numbers = generate_seed_numbers(100000)

    sample_index = 0
    total_attempt_count = -1
    attempt_count_per_sample = -1

    while sample_index < num_sample_per_env:
        total_attempt_count += 1
        attempt_count_per_sample += 1
        if total_attempt_count >= 100000:
            raise ValueError("Exceeded the maximum number of attempts.")
        env_noised = copy.deepcopy(env)
        np.random.seed(seed_numbers[total_attempt_count])
        num_choice = np.max((int(np.ceil(len(env_noised.nonblocks) * max_ratio)) - (attempt_count_per_sample // 100), 1))
        noise_blocks = np.random.choice(
            env_noised.nonblocks,
            num_choice,
            replace=False,
        )
        env_noised.update_map(blocks=np.append(env_noised.blocks, noise_blocks))
        if not env_noised.is_connected():
            continue
        
        (
            vertex_nodes_noised,
            deadend_nodes_noised,
            edge_nodes_noised,
            G_noised,
            image_noised,
            skeleton_noised,
            vertex_corresp_noised,
            deadend_corresp_noised,
            edge_corresp_noised,
            _, _, _
        ) = skeletonize_env(env_noised)
        
        if G_noised is None:
            continue
        matcher = nx.algorithms.isomorphism.GraphMatcher(G, G_noised)
        if matcher.is_isomorphic():
            perm = matcher.mapping
        else:
            continue

        env_list.append(env_noised)
        vertex_nodes_list.append(vertex_nodes_noised)
        deadend_nodes_list.append(deadend_nodes_noised)
        edge_nodes_list.append(edge_nodes_noised)

        blocks = env_noised.return_blocks()
        srmb_noised = SRMB(env_noised.num_row, env_noised.num_column)
        srmb_noised.update_map_structure(blocks=blocks)
        eigenmap_noised = srmb_noised.return_successormap()

        (
            mtx_original,
            mtx_transformed,
            mtx_stand,
            grid_cost_pot,
            grid_cost_disp,
            best_sign_configuration,
            best_optimal_rotation_matrix,
            scale,
            scale_noised,
        ) = calc_ot_post_procrustes(eigenmap, eigenmap_noised)

        mtx_original_list.append(center_and_scale_diffusionmap(mtx_stand[env_noised.nonblocks, :3].real)[0])
        mtx_transformed_list.append(center_and_scale_diffusionmap(mtx_transformed[env_noised.nonblocks, :3].real)[0])

        sample_index += 1
        attempt_count_per_sample = -1

    return mtx_original_list, mtx_transformed_list, vertex_nodes_list, deadend_nodes_list, edge_nodes_list, env_list

env = MazeVisualization(7, 7)
nonblocks = [3, 10, 16, 17, 18, 21, 22, 23, 25, 26, 27, 30, 31, 32]
env.set_start_goal_states(16, 3)
env.update_map(blocks=np.setdiff1d(np.arange(env.num_states), nonblocks))

mtx_original_list_shift, mtx_transformed_list_shift, vertex_nodes_list_shift, deadend_nodes_list_shift, edge_nodes_list_shift = process_shift(env)
mtx_original_list_scale, mtx_transformed_list_scale, vertex_nodes_list_scale, deadend_nodes_list_scale, edge_nodes_list_scale = process_scale(env, 5)
mtx_original_list_rotation, mtx_transformed_list_rotation, vertex_nodes_list_rotation, deadend_nodes_list_rotation, edge_nodes_list_rotation = process_rotation(env)
mtx_original_list_noise, mtx_transformed_list_noise, vertex_nodes_list_noise, deadend_nodes_list_noise, edge_nodes_list_noise, env_noise = process_noise(env, 1)

In [4]:
from utils.plot import scatter, scatter_multiple
color = "#B0B0B0"

In [5]:
num_row, num_column = 5, 5
figsize_x = np.min([num_column * subplot_scale_factor, 20])
figsize_y = figsize_x * num_row / num_column
fig = plt.figure(figsize=(figsize_x + 4, figsize_y))
gs = gridspec.GridSpec(num_row, num_column, figure=fig, height_ratios=[1, 1, 1, 1, 1.5], width_ratios=[1 for _ in range(num_column)])

# 개별 subplot 생성
ax_original = fig.add_subplot(gs[0, 0])
ax_shift = fig.add_subplot(gs[0, 2])
ax_scale = fig.add_subplot(gs[0, 3])
ax_rotation = fig.add_subplot(gs[0, 4])
ax_noise = fig.add_subplot(gs[0, 1])
ax_original_skel = fig.add_subplot(gs[1:3, 0])
ax_shift_skel_before = fig.add_subplot(gs[1, 2])
ax_shift_skel_after = fig.add_subplot(gs[2, 2])
ax_scale_skel_before = fig.add_subplot(gs[1, 3])
ax_scale_skel_after = fig.add_subplot(gs[2, 3])
ax_rotation_skel_before = fig.add_subplot(gs[1, 4])
ax_rotation_skel_after = fig.add_subplot(gs[2, 4])
ax_noise_skel_before = fig.add_subplot(gs[1, 1])
ax_noise_skel_after = fig.add_subplot(gs[2, 1])

ax_original_fiedler = fig.add_subplot(gs[3, 0])
ax_shift_fiedler = fig.add_subplot(gs[3, 2])
ax_scale_fiedler = fig.add_subplot(gs[3, 3])
ax_rotation_fiedler = fig.add_subplot(gs[3, 4])
ax_noise_fiedler = fig.add_subplot(gs[3, 1])

ax_invariance_summary = fig.add_subplot(gs[4, 0])
ax_invariance = fig.add_subplot(gs[4, 1:3])
ax_shift_invariance = fig.add_subplot(gs[4, 3])
ax_scale_invariance = fig.add_subplot(gs[4, 4])

def flatten(nested_list):
    flat_list = []
    for item in nested_list:
        if isinstance(item, (list, tuple)):
            flat_list.extend(flatten(item))
        else:
            flat_list.append(item)
    return flat_list

env.visualize(
    display=True, no_startgoal=True, ax=ax_original, set_grid=True,
    vertex_states=flatten(vertex_nodes_list_shift[0]),
    deadend_states=flatten(deadend_nodes_list_shift[0]),
    edge_states=flatten(edge_nodes_list_shift[0]),
    junction_color=junction_color, deadend_color=deadend_color, edge_color=edge_color
)
ax_original.set_title('Original', fontsize=title_font)

shifted_env = copy.deepcopy(env)
scaled_env = copy.deepcopy(env)
rotated_env = copy.deepcopy(env)

shifted_env.shift_map_w_expansion((2, 2))
scaled_env.scale_map(2)
rotated_env.rotate_map(1)

vertex_nodes_shift_5 = [shifted_env.index_to_state(4, 5), shifted_env.index_to_state(5, 4), shifted_env.index_to_state(5, 6)]
deadend_nodes_shift_5 = [shifted_env.index_to_state(0, 5), shifted_env.index_to_state(5, 0), shifted_env.index_to_state(5, 8)]
combined = np.concatenate([vertex_nodes_shift_5, deadend_nodes_shift_5, shifted_env.blocks])
edge_nodes_shift_5 = np.setdiff1d(np.arange(shifted_env.num_states), combined)

vertex_nodes_shift_5_plot = [[shifted_env.index_to_state(4, 5)], [shifted_env.index_to_state(5, 4)], [shifted_env.index_to_state(5, 6)]]
deadend_nodes_shift_5_plot = [[shifted_env.index_to_state(0, 5)], [shifted_env.index_to_state(5, 0)], [shifted_env.index_to_state(5, 8)]]
edge_nodes_shift_5_plot = [edge_nodes_shift_5]

shifted_env.visualize(
    display=True, no_startgoal=True, ax=ax_shift, set_grid=True,
    vertex_states=vertex_nodes_shift_5,
    deadend_states=deadend_nodes_shift_5,
    edge_states=edge_nodes_shift_5,
    junction_color=junction_color, deadend_color=deadend_color, edge_color=edge_color
)
ax_shift.set_title('Shift', fontsize=title_font)

scaled_env.visualize(
    display=True, no_startgoal=False, ax=ax_scale, set_grid=True,
    vertex_states=flatten(vertex_nodes_list_scale[1]),
    deadend_states=flatten(deadend_nodes_list_scale[1]),
    edge_states=flatten(edge_nodes_list_scale[1]),
    junction_color=junction_color, deadend_color=deadend_color, edge_color=edge_color
)
ax_scale.yaxis.set_major_locator(MultipleLocator(1))
ax_scale.xaxis.set_major_locator(MultipleLocator(1))
ax_scale.set_title('Scale', fontsize=title_font)

rotated_env.visualize(
    display=True, no_startgoal=True, ax=ax_rotation, set_grid=True,
    vertex_states=flatten(vertex_nodes_list_rotation[1]),
    deadend_states=flatten(deadend_nodes_list_rotation[1]),
    edge_states=flatten(edge_nodes_list_rotation[1]),
    junction_color=junction_color, deadend_color=deadend_color, edge_color=edge_color
)
ax_rotation.set_title('Rotation', fontsize=title_font)

noised_env = MazeVisualization(env.num_row, env.num_column)
noised_env.update_map(blocks=env_noise[0].blocks)
noised_env.visualize(
    display=True, no_startgoal=True, ax=ax_noise, set_grid=True,
    vertex_states=flatten(vertex_nodes_list_noise[0]),
    deadend_states=flatten(deadend_nodes_list_noise[0]),
    edge_states=flatten(edge_nodes_list_noise[0]),
    junction_color=junction_color, deadend_color=deadend_color, edge_color=edge_color
)
ax_noise.set_title('Noise', fontsize=title_font)



(
    vertex_nodes,
    deadend_nodes,
    edge_nodes,
    G,
    image,
    skeleton,
    vertex_corresp,
    deadend_corresp,
    edge_corresp,
    _, _, _
) = skeletonize_env(env)

'''draw_graph_with_curved_edges(G, "", save=False, ax=ax_topology, rad=1, node_size=node_size, edge_width=edge_width)
ax_topology.set_title('Topology', fontsize=title_font)'''

dim = 2
mtx_original = copy.deepcopy(mtx_original_list_shift[0])
mtx_original[:, 1] *= -1
mtx_original_shift = copy.deepcopy(mtx_original_list_shift[5])
mtx_original_shift[:, 0] *= -1
mtx_original_shift[:, 1] *= -1
mtx_transformed_shift = copy.deepcopy(mtx_transformed_list_shift[5])
mtx_transformed_shift[:, 0] *= -1
mtx_transformed_shift[:, 1] *= -1
mtx_original_scale = copy.deepcopy(mtx_original_list_scale[1])
mtx_original_scale[:, 1] *= -1
mtx_transformed_scale = copy.deepcopy(mtx_transformed_list_scale[1])
mtx_transformed_scale[:, 1] *= -1
mtx_original_rotation = copy.deepcopy(mtx_original_list_rotation[1])
mtx_original_rotation[:, 1] *= -1
mtx_transformed_rotation = copy.deepcopy(mtx_transformed_list_rotation[1])
mtx_transformed_rotation[:, 1] *= -1
mtx_original_noise = copy.deepcopy(mtx_original_list_noise[0])
mtx_original_noise[:, 1] *= -1
mtx_transformed_noise = copy.deepcopy(mtx_transformed_list_noise[0])
mtx_transformed_noise[:, 1] *= -1

draw_scatter_plot_skeleton(
    mtx_original[:, :dim],
    vertex_nodes, deadend_nodes, edge_nodes,
    env.nonblocks, num_row=env.num_row, num_column=env.num_column,
    type="eigenmap", sizes_per_type=[node_size, node_size-100, node_size-200],
    ax=ax_original_skel, fig=fig, axis_off=False,
)
draw_scatter_plot_skeleton(
    mtx_original_shift[:, :dim],
    vertex_nodes_shift_5_plot, deadend_nodes_shift_5_plot, edge_nodes_shift_5_plot,
    shifted_env.nonblocks, num_row=shifted_env.num_row, num_column=shifted_env.num_column,
    type="eigenmap", sizes_per_type=[node_size, node_size-100, node_size-200],
    ax=ax_shift_skel_before, fig=fig, axis_off=False,
)
draw_scatter_plot_skeleton(
    mtx_transformed_shift[:, :dim],
    vertex_nodes_shift_5_plot, deadend_nodes_shift_5_plot, edge_nodes_shift_5_plot,
    shifted_env.nonblocks, num_row=shifted_env.num_row, num_column=shifted_env.num_column,
    type="eigenmap", sizes_per_type=[node_size, node_size-100, node_size-200],
    ax=ax_shift_skel_after, fig=fig, axis_off=False,
)
draw_scatter_plot_skeleton(
    mtx_original_scale[:, :dim],
    vertex_nodes_list_scale[1], deadend_nodes_list_scale[1], edge_nodes_list_scale[1],
    scaled_env.nonblocks, num_row=scaled_env.num_row, num_column=scaled_env.num_column,
    type="eigenmap", sizes_per_type=[node_size, node_size-100, node_size-200],
    ax=ax_scale_skel_before, fig=fig, axis_off=False,
)
draw_scatter_plot_skeleton(
    mtx_transformed_scale[:, :dim],
    vertex_nodes_list_scale[1], deadend_nodes_list_scale[1], edge_nodes_list_scale[1],
    scaled_env.nonblocks, num_row=scaled_env.num_row, num_column=scaled_env.num_column,
    type="eigenmap", sizes_per_type=[node_size, node_size-100, node_size-200],
    ax=ax_scale_skel_after, fig=fig, axis_off=False,
)
draw_scatter_plot_skeleton(
    mtx_original_rotation[:, :dim],
    vertex_nodes_list_rotation[1], deadend_nodes_list_rotation[1], edge_nodes_list_rotation[1],
    rotated_env.nonblocks, num_row=rotated_env.num_row, num_column=rotated_env.num_column,
    type="eigenmap", sizes_per_type=[node_size, node_size-100, node_size-200],
    ax=ax_rotation_skel_before, fig=fig, axis_off=False,
)
draw_scatter_plot_skeleton(
    mtx_transformed_rotation[:, :dim],
    vertex_nodes_list_rotation[1], deadend_nodes_list_rotation[1], edge_nodes_list_rotation[1],
    rotated_env.nonblocks, num_row=rotated_env.num_row, num_column=rotated_env.num_column,
    type="eigenmap", sizes_per_type=[node_size, node_size-100, node_size-200],
    ax=ax_rotation_skel_after, fig=fig, axis_off=False,
)
draw_scatter_plot_skeleton(
    mtx_original_noise[:, :dim],
    vertex_nodes_list_noise, deadend_nodes_list_noise, edge_nodes_list_noise,
    noised_env.nonblocks, num_row=noised_env.num_row, num_column=noised_env.num_column,
    type="eigenmap", sizes_per_type=[node_size, node_size-100, node_size-200],
    ax=ax_noise_skel_before, fig=fig, axis_off=False,
)
draw_scatter_plot_skeleton(
    mtx_transformed_noise[:, :dim],
    vertex_nodes_list_noise, deadend_nodes_list_noise, edge_nodes_list_noise,
    noised_env.nonblocks, num_row=noised_env.num_row, num_column=noised_env.num_column,
    type="eigenmap", sizes_per_type=[node_size, node_size-100, node_size-200],
    ax=ax_noise_skel_after, fig=fig, axis_off=False,
)

axes = [
    ax_original_skel, ax_shift_skel_before, ax_shift_skel_after,
    ax_scale_skel_before, ax_scale_skel_after, ax_rotation_skel_before,
    ax_rotation_skel_after, ax_noise_skel_before, ax_noise_skel_after
]

# Loop through each axis and set the properties
for ax in axes:
    ax.set_xlim(-0.55, 0.55)
    ax.set_ylim(-0.55, 0.55)
    ax.set_aspect('equal', 'box')
    ax.set_xticks([])
    ax.set_yticks([])

# Assuming the data is already loaded as per the provided code
shift_group_means = np.load("data/robustness/shift/skel_grid_cost_gw_mean.npy")[1:][::-1]
shift_group_stds = np.load("data/robustness/shift/skel_grid_cost_gw_std.npy")[1:][::-1]
scale_group_means = np.load(
    "data/robustness/scale/skel_grid_cost_normalized_gw_mean.npy"
)[1:][::-1]
scale_group_stds = np.load("data/robustness/scale/skel_grid_cost_normalized_gw_std.npy")[1:][::-1]
rotation_group_means = np.load(
    "data/robustness/rotation/skel_grid_cost_gw_mean.npy"
)[1:]
'''rotation_group_sems = np.load("data/robustness/rotation/skel_grid_cost_gw_sem.npy")[
    1:
]'''
rotation_group_stds = np.load("data/robustness/rotation/skel_grid_cost_gw_std.npy")[
    1:
]
noise_group_means = np.array(
    [np.load("data/robustness/noise/skel_grid_cost_gw_mean.npy")]
)
'''noise_group_sems = np.array(
    [np.load("data/robustness/noise/skel_grid_cost_gw_sem.npy")]
)'''
noise_group_stds = np.array(
    [np.load("data/robustness/noise/skel_grid_cost_gw_std.npy")]
)
baseline_group_mean, baseline_group_std = postprocess_all_skel_gw_nonisomorphic()
tag_group_mean, tag_group_std = postprocess_all_skel_gw()




# Data for plotting
means = [baseline_group_mean, tag_group_mean]
stds = [baseline_group_std, tag_group_std]
categories = ["Non-isomorphic pair", "Isomorphic augmentation"]
x = np.arange(len(categories))


# Plot bars with black borders
bars = ax_invariance_summary.bar(x, means, yerr=stds, color=color, edgecolor="black", width=0.6, error_kw={'elinewidth': 2})

# Set labels and ticks
ax_invariance_summary.set_ylabel("Grid code distance (GW distance)", fontsize=axis_font)
ax_invariance_summary.set_xticks(x)
ax_invariance_summary.set_xticklabels(categories, fontsize=tick_font)
ax_invariance_summary.set_ylim(0, 0.035)
ax_invariance_summary.tick_params(axis='x', labelsize=tick_font)
ax_invariance_summary.tick_params(axis='y', labelsize=tick_font)
ax_invariance_summary.spines['top'].set_visible(False)
ax_invariance_summary.spines['right'].set_visible(False)


means = [
    [baseline_group_mean],
    noise_group_means,
    shift_group_means,
    scale_group_means,
    rotation_group_means,
]

stds = [
    [baseline_group_std],
    noise_group_stds,
    shift_group_stds,
    scale_group_stds,
    rotation_group_stds,
]

# Bar plot
labels = ["Baseline (Non-isomorphic)", "Noise", "Shift", "Scale", "Rotation"]
width = 0.2  # the width of the bars

# Calculate the maximum number of bars in any group
max_bars = max(len(group) for group in means)

x_positions = []
current_position = 0
for i in range(len(means)):
    x_positions.append(np.arange(len(means[i])) * (width + 0.05) + current_position)
    current_position += (
        len(means[i]) * (width + 0.05) + width / 2
    )  # Reduced extra space between different types

color_perm = {
    0: 4,
    1: 2,
    2: 1,
    3: 0,
    4: 3,
}

# Plotting each group of bars
for i in range(1, len(means)):
    ax_invariance.bar(
        x_positions[i],
        means[i],
        width,
        yerr=stds[i],
        label=labels[i],
        # color=plt.cm.tab10(color_perm[i]),
        color=palette[i],
        edgecolor="black",
    )

# Add some text for labels, title and custom x-axis tick labels, etc.
ax_invariance.set_xlabel("Augmentation type and level", fontsize=axis_font)
ax_invariance.set_ylabel("Grid code distance (GW distance)", fontsize=axis_font)
# ax.set_title("Grid Cost GW by Invariance Type")

# Custom x-axis tick labels for each bar

custom_labels = [
    [""],
    [""],
    [r"$2\sqrt{2}$", r"$\sqrt{5}$", "2", r"$\sqrt{2}$", "1"],
    ["5x", "4x", "3x", "2x"],
    [r"$90^\circ$", r"$180^\circ$", r"$270^\circ$"],
]

ax_invariance.axhline(y=baseline_group_mean, lw=2, color='black', linestyle='-', label='Baseline (Non-isomorphic)')


# Flatten the custom_labels list and set the x-ticks accordingly
flat_labels = [item for sublist in custom_labels[1:] for item in sublist]
flat_x_positions = [pos for sublist in x_positions[1:] for pos in sublist]

ax_invariance.set_xticks(flat_x_positions)
ax_invariance.set_xticklabels(flat_labels, fontsize=tick_font)
ax_invariance.set_yticks([0, 0.01, 0.02, 0.03])
ax_invariance.set_yticklabels([0, 0.01, 0.02, 0.03], fontsize=tick_font)
ax_invariance.legend(fontsize=legend_font, loc='upper right', frameon=False)
ax_invariance.tick_params(axis='x', labelsize=tick_font)
ax_invariance.tick_params(axis='y', labelsize=tick_font)
ax_invariance.set_ylim(0, 0.02)
ax_invariance.spines['top'].set_visible(False)
ax_invariance.spines['right'].set_visible(False)


# Load the data for grid_cost_ot and grid_cost_pot for shift
shift_group_ot_means = np.load("data/robustness/shift/skel_grid_cost_ot_mean.npy")[1:][::-1]
shift_group_ot_stds = np.load("data/robustness/shift/skel_grid_cost_ot_std.npy")[1:][::-1]
shift_group_pot_means = np.load("data/robustness/shift/skel_grid_cost_pot_mean.npy")[
    1:
][::-1]
shift_group_pot_stds = np.load("data/robustness/shift/skel_grid_cost_pot_std.npy")[
    1:
][::-1]

# Bar plot
labels = ["1", r"$\sqrt{2}$", "2", r"$\sqrt{5}$", r"$2\sqrt{2}$"][::-1]
width = 0.35  # the width of the bars

# Calculate x positions for the bars
x_positions = []
current_position = 0
for i in range(len(labels)):
    x_positions.append([current_position, current_position + width])
    current_position += (
        2 * width + width / 2
    )  # Reduced extra space between different types

# Flatten the x_positions list
flat_x_positions = [pos for sublist in x_positions for pos in sublist]

# Plotting the bars for grid_cost_ot and grid_cost_pot
darker_color = sns.color_palette("tab10")[1]
darker_color = (darker_color[0] * 0.6, darker_color[1] * 0.6, darker_color[2] * 0.6)

ax_shift_invariance.bar(
    flat_x_positions[::2],
    shift_group_ot_means,
    width,
    yerr=shift_group_ot_stds,
    label="Without rotation",
    color=darker_color,  # Darker color
    edgecolor="black",
)
ax_shift_invariance.bar(
    flat_x_positions[1::2],
    shift_group_pot_means,
    width,
    yerr=shift_group_pot_stds,
    label="With rotation",
    color=sns.color_palette("tab10")[1],  # Original color
    edgecolor="black",
)

# Add some text for labels, title and custom x-axis tick labels, etc.
# ax.set_ylim(0, 0.1)
ax_shift_invariance.set_xlabel("Shift", fontsize=axis_font)
ax_shift_invariance.set_ylabel("Grid code distance (Wasserstein distance)", fontsize=axis_font)
# ax.set_title("Comparison of Grid Cost OT and POT for Shift")
ax_shift_invariance.set_xticks([np.mean(x) for x in x_positions])
ax_shift_invariance.set_xticklabels(labels)
ax_shift_invariance.set_yticks([0, 0.1, 0.2])
ax_shift_invariance.set_yticklabels([0, 0.1, 0.2], fontsize=tick_font)
ax_shift_invariance.tick_params(axis='x', labelsize=tick_font, length=0)
ax_shift_invariance.tick_params(axis='y', labelsize=tick_font)
ax_shift_invariance.legend(fontsize=legend_font, frameon=False)
ax_shift_invariance.set_ylim(0, 0.25)
ax_shift_invariance.spines['top'].set_visible(False)
ax_shift_invariance.spines['right'].set_visible(False)

original_srmb = SRMB(env.num_row, env.num_column)
original_srmb.update_map_structure(env.blocks)
original_diffusionmap = original_srmb.return_successormap()
original_fiedler = original_diffusionmap[:, 0]
for b in env.blocks:
    original_fiedler[b] = np.nan
shifted_srmb = SRMB(shifted_env.num_row, shifted_env.num_column)
shifted_srmb.update_map_structure(shifted_env.blocks)
shifted_diffusionmap = shifted_srmb.return_successormap()
shifted_fiedler = shifted_diffusionmap[:, 0]
for b in shifted_env.blocks:
    shifted_fiedler[b] = np.nan
scaled_srmb = SRMB(scaled_env.num_row, scaled_env.num_column)
scaled_srmb.update_map_structure(scaled_env.blocks)
scaled_diffusionmap = scaled_srmb.return_successormap()
scaled_fiedler = scaled_diffusionmap[:, 0]
for b in scaled_env.blocks:
    scaled_fiedler[b] = np.nan
rotated_srmb = SRMB(rotated_env.num_row, rotated_env.num_column)
rotated_srmb.update_map_structure(rotated_env.blocks)
rotated_diffusionmap = rotated_srmb.return_successormap()
rotated_fiedler = rotated_diffusionmap[:, 0]
for b in rotated_env.blocks:
    rotated_fiedler[b] = np.nan
noise_srmb = SRMB(env.num_row, env.num_column)
noise_srmb.update_map_structure(env_noise[0].blocks)
noise_diffusionmap = noise_srmb.return_successormap()
noise_fiedler = noise_diffusionmap[:, 0]
for b in env_noise[0].blocks:
    noise_fiedler[b] = np.nan

cmap = matplotlib.colormaps.get_cmap('jet')
cmap.set_bad('black')
ax_original_fiedler.matshow(original_fiedler.reshape(env.num_row, env.num_column) * -1, cmap=cmap)
ax_original_fiedler.tick_params(
    left=False,
    right=False,
    labelleft=False,
    labelbottom=False,
    bottom=False,
    top=False,
    labeltop=False,
)
ax_shift_fiedler.matshow(shifted_fiedler.reshape(shifted_env.num_row, shifted_env.num_column) * -1, cmap=cmap)
ax_shift_fiedler.tick_params(
    left=False,
    right=False,
    labelleft=False,
    labelbottom=False,
    bottom=False,
    top=False,
    labeltop=False,
)
ax_scale_fiedler.matshow(scaled_fiedler.reshape(scaled_env.num_row, scaled_env.num_column), cmap=cmap)
ax_scale_fiedler.tick_params(
    left=False,
    right=False,
    labelleft=False,
    labelbottom=False,
    bottom=False,
    top=False,
    labeltop=False,
)
ax_rotation_fiedler.matshow(rotated_fiedler.reshape(rotated_env.num_row, rotated_env.num_column), cmap=cmap)
ax_rotation_fiedler.tick_params(
    left=False,
    right=False,
    labelleft=False,
    labelbottom=False,
    bottom=False,
    top=False,
    labeltop=False,
)
ax_noise_fiedler.matshow(noise_fiedler.reshape(env.num_row, env.num_column), cmap=cmap)
ax_noise_fiedler.tick_params(
    left=False,
    right=False,
    labelleft=False,
    labelbottom=False,
    bottom=False,
    top=False,
    labeltop=False,
)

# Bar plot
labels = ["2x", "3x", "4x", "5x"][::-1]
width = 0.35  # the width of the bars
# Calculate x positions for the bars
x_positions = []
current_position = 0
for i in range(len(labels)):
    x_positions.append([current_position, current_position + width])
    current_position += (
        2 * width + width / 2
    )  # Reduced extra space between different types

# Flatten the x_positions list
flat_x_positions = [pos for sublist in x_positions for pos in sublist]

# Plotting the bars for grid_cost_gw and grid_cost_normalized_gw
# Darken the color for "Without Scaling"
darker_color = sns.color_palette("tab10")[0]
darker_color = (darker_color[0] * 0.6, darker_color[1] * 0.6, darker_color[2] * 0.6)

# Load the data for grid_cost_gw and grid_cost_normalized_gw for scale
scale_group_gw_means = np.load("data/robustness/scale/skel_grid_cost_gw_mean.npy")[1:][::-1]
scale_group_gw_stds = np.load("data/robustness/scale/skel_grid_cost_gw_std.npy")[1:][::-1]
scale_group_normalized_gw_means = np.load(
    "data/robustness/scale/skel_grid_cost_normalized_gw_mean.npy"
)[1:][::-1]
scale_group_normalized_gw_stds = np.load(
    "data/robustness/scale/skel_grid_cost_normalized_gw_std.npy"
)[1:][::-1]



# Plotting the bars for grid_cost_gw and grid_cost_normalized_gw
ax_scale_invariance.bar(
    flat_x_positions[::2],
    scale_group_gw_means,
    width,
    yerr=scale_group_gw_stds,
    label="Without scaling",
    color=darker_color,  # Darker color
    edgecolor="black",
)
ax_scale_invariance.bar(
    flat_x_positions[1::2],
    scale_group_normalized_gw_means,
    width,
    yerr=scale_group_normalized_gw_stds,
    label="With scaling",
    color=sns.color_palette("tab10")[0],  # Original color
    edgecolor="black",
)

# Add some text for labels, title and custom x-axis tick labels, etc.
ax_scale_invariance.set_xlabel("Scale", fontsize=axis_font)
ax_scale_invariance.set_ylabel("Grid code distance (GW distance)", fontsize=axis_font)
# ax.set_title("Comparison of Grid Cost GW and Normalized GW for Scale")
ax_scale_invariance.set_xticks([np.mean(x) for x in x_positions])
ax_scale_invariance.set_xticklabels(labels)
ax_scale_invariance.set_yticks([0, 1, 2, 3])
ax_scale_invariance.set_yticklabels([0, 1, 2, 3], fontsize=tick_font)
ax_scale_invariance.tick_params(axis='x', labelsize=tick_font, length=0)
ax_scale_invariance.tick_params(axis='y', labelsize=tick_font)
ax_scale_invariance.spines['top'].set_visible(False)
ax_scale_invariance.spines['right'].set_visible(False)

ax_scale_invariance.legend(fontsize=legend_font, frameon=False)

# Remove the grid
ax_scale_invariance.grid(False)

# Set black border
for spine in ax_scale_invariance.spines.values():
    spine.set_edgecolor("black")
# Set outer ticks
ax_scale_invariance.set_yticks(ax_scale_invariance.get_yticks())  # Use the natural y-tick positions
ax_scale_invariance.set_yticklabels(ax_scale_invariance.get_yticklabels())  # Use the natural y-tick labels
ax_scale_invariance.set_ylim(0, 3)

fig.tight_layout()
plt.savefig('./figure/figure_4.svg', bbox_inches='tight', transparent=True)
plt.show()